# Session de Recrutement Printemps 2017 - Team Tax Analytics
## Rendu de Guillaume Grelet - guillaume.grelet@telecom-paristech.fr

Date : février 2017


Ce rendu a pour but de fournir une réponse à l'exercice posé par PwC sur la gestion de tableur Excel.  
Il utilise les packages suivante : jupyter, pandas, PyQt, ipywidgets  
  
  
Notes :
Le code suivant est systématiquement commenté en anglais, comme l'usage le veut.  


In [200]:
# Import the necessary packages..

# For the widgets
%gui qt 
from IPython.display import display, Markdown, Latex
from PyQt5.QtWidgets import QFileDialog
from ipywidgets import *

# For the excel file processing...
import pandas as pd
import numpy as np
import math

import jellyfish

In [19]:
def filesearch(dir=None):
    """
    That function allows the user to select a file via a PyQt5 dialog window and return the file name as a str
    """
    if dir is None: dir ="~/" # Default path is ~/
    filename = QFileDialog.getOpenFileName(None)
    return filename[0]

In [20]:
datapath = filesearch("~/")

In [21]:
%%time
print("Processing file...")
data = pd.read_excel(datapath)
print("\nProcessing OK. Stats about processing:\n")

Processing file...

Processing OK. Stats about processing:

CPU times: user 6.76 s, sys: 148 ms, total: 6.91 s
Wall time: 6.97 s


In [22]:
input_label_widget = widgets.Textarea(
    placeholder='Place here the preprocessed labels',
    value='code_hospidiag\nsous_axe',
    disabled=False
)

button1 = widgets.Button(
    description='Save',
    disabled=False,
    button_style='info', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click me',
    icon='check'
)


display(Markdown("## Remplissez ci-dessus la liste des labels standards préprogrammés  "), input_label_widget, button1)

def handle_submit(sender):
    global x
    x = input_label_widget.value
    global input_label_list
    input_label_list = x.split()
    print("input_label_list = ")
    print(input_label_list)
    return None

def when_button_clicked(button1):
    out = handle_submit(input_label_widget)
    return None 

button1.on_click(when_button_clicked)

## Remplissez ci-dessus la liste des labels standards préprogrammés  

input_label_list = 
['code_hospidiag', 'sous_axe']


In [23]:
found_labels = list(data)
print("Found labels are the following:")
for str in found_labels:
    print("    " + str)

Found labels are the following:
    axe
    sous_axe
    indic
    code_hospidiag
    libelle


In [24]:
intersection = list(set(found_labels).intersection(input_label_list))
#print(intersection)

existing_label_widget = widgets.Textarea(
    value="\n".join(intersection) + "\n",
    placeholder='Type something',
    disabled=True
)

display(Markdown("## Les labels ci-dessus sont issus du fichier Excel et existent dans la liste prépprogrammée"), existing_label_widget)

## Les labels ci-dessus sont issus du fichier Excel et existent dans la liste prépprogrammée

In [25]:
new_labels = []
for str in found_labels:
    if str not in intersection:
        new_labels.append(str)
                
t = widgets.Textarea(
    value="\n".join(new_labels) + "\n",
    placeholder='Type something',
    disabled=False
)

b = widgets.Button(
    description='Ok',
    disabled=False,
    button_style='info', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click me',
    icon='check'
)

display(Markdown("## Les labels ci-dessus ne sont pas dans la liste standardisée et sont donc nouveaux. Vous pouvez modifier la liste et les ajouter"), t, b)

def handle_submit(sender):
    global y
    y = t.value
    global modified_list
    modified_list = y.split()
    print("modified_list = ")
    print(modified_list)
    return None

def when_button_clicked(button):
    out = handle_submit(t)
    return None 

b.on_click(when_button_clicked)

## Les labels ci-dessus ne sont pas dans la liste standardisée et sont donc nouveaux. Vous pouvez modifier la liste et les ajouter

modified_list = 
['axe', 'indic', 'libelle']
modified_list = 
['axer', 'indicquer', 'libbele']


In [53]:
jellyfish.jaro_distance('jellyfish', 'smellyfish')

0.8962962962962964

In [210]:
mat_size = (len(modified_list) + len(input_label_list)), (len(found_labels))
mat0 = np.array([modified_list + input_label_list])

mat1 = np.array([[None] +  modified_list + input_label_list])
mat2 = np.array([found_labels]).T
mat3 = np.zeros(mat_size)
a, b = 0, 0
for str1 in modified_list + input_label_list:
    for str2 in found_labels:
        mat3[a, b] = round(jellyfish.jaro_distance(str1, str2), 3)
        a = a + 1
    a = 0
    b = b + 1
mat4 = np.concatenate((mat2, mat3), axis = 1)
mat5 = np.concatenate((mat1, mat4), axis = 0)

In [212]:
print(mat)

[[None 'axer' 'indicquer' 'libbele' 'code_hospidiag' 'sous_axe']
 ['axe' '0.917' '0.0' '0.492' '0.468' '0.0']
 ['sous_axe' '0.0' '0.412' '0.423' '0.512' '1.0']
 ['indic' '0.0' '0.852' '0.448' '0.494' '0.0']
 ['code_hospidiag' '0.44' '0.41' '0.405' '1.0' '0.512']
 ['libelle' '0.464' '0.503' '0.905' '0.405' '0.423']]


In [217]:
treshold = 0.8
k = 0
a = np.where(mat3>treshold)

while k < np.maximum(len(a[0]),len(a[1])):
    print(mat2[a[0][k], 0] + " ressemble à " + mat0[0,a[1][k]])
    k = k + 1

axe ressemble à axer
sous_axe ressemble à sous_axe
indic ressemble à indicquer
code_hospidiag ressemble à code_hospidiag
libelle ressemble à libbele


In [138]:
'''md_str = "# Would you like to keep the following new labels ? You can also edit them before saving them. Just run the next input"
new_label_str = "[\"" + "\", \"".join(new_labels) + "\"]"

input_str = "label_list = input_label_list + " + new_label_str

display(Markdown(md_str))
get_ipython().set_next_input(input_str)'''

['code_hospidiag', 'libelle']